<p align="left">
  <img src="./logo_UTN.svg" />
</p>


# **Teoría de Circuitos II - R4001 - 2023**
## Trabajo Semanal 2
### Autor: Bruno Glecer

## Consigna

La tarea semanal consiste en realizar un analisis del siguiente circuito

<div>
    <img src="circuito.png" width="1000"/>
</div>

Se piden las siguientes consignas:

1. Hallar la transferencia $T = \frac{V_o}{V_i}$ en función de $\omega_0$ y $Q$
2. Obtener el valor de los componentes del circuito de forma tal que $\omega_0 = 1$ y $Q = 3$
3. Ajustar el valor de $R_1$ de forma tal que $\left|T(0)\right| = 20 db$
4. (Bonus) Obtener los valores de la red normalizados en frecuencia e impedancia.
5. (TODO) (Bonus) Calcular las sensibilidades $S^{\omega_0}_C$, $S^{Q}_{R_2}$ y $S^{Q}_{R_3}$.
6. (TODO) (Bonus) Recalcular los valores de la red para que cumpla con una transferencia Butterworth.
7. (TODO) (Bonus) Cómo podría obtener un circuito pasabanda con los mismos componentes originales y con qué parámetros quedaría diseñado (Ver ejemplo 4.6 en Schaumann).
8. (TODO) (Bonus) Simulación circuital de todos los experimentos.


## 1. Deducción de la función de transferencia

Para simplificar el análisis se asumirá que los op-amps se encuentran en equilibrio y su ganancia es arbitrariamente alta. En otras palabras la tensión de entrada diferencial se asumirá que es cero.

Primero, le asignamos nombre a todas las corrientes y tensiones desconocidas en el circuito.

<div>
    <img src="circuito_anotado.png" width="800"/>
</div>

Por ley de nodos, tenemos:

$$I_1 + I_6 = I_2 + I_3$$

Seguimos haciendo los remplazos $I_1 = \frac{V_i}{R_1}$, $I_6 = \frac{V_o}{R_3}$, $I_2 = -\frac{V_a}{R_2}$ y $I_3 = - V_a C s$ para obtener:

$$\frac{V_i}{R_1} + \frac{V_o}{R_3} = -V_a\left(\frac{1}{R_2} + C s \right)$$

Remplazando $V_a = I_4 R_3$

$$\frac{V_i}{R_1} + \frac{V_o}{R_3} = -I_4 R_3\left(\frac{1}{R_2} + C s \right)$$

Remplazando $I_4 = - V_b C s$

$$\frac{V_i}{R_1} + \frac{V_o}{R_3} = V_b R_3 C s\left(\frac{1}{R_2} + C s \right)$$

Remplazando $V_b = I_5 R_4$

$$\frac{V_i}{R_1} + \frac{V_o}{R_3} = I_5 R_4 R_3 C s\left(\frac{1}{R_2} + C s \right)$$

Remplazando $I_5 = -\frac{V_o}{R_4}$

$$\frac{V_i}{R_1} + \frac{V_o}{R_3} = - V_o R_3 C s\left(\frac{1}{R_2} + C s \right)$$

A partir de aca se puede reordenar para encontrar $T = \frac{V_o}{V_i}$

$$T(s) = \frac{V_o}{V_i} = - \frac{1}{C^2 R_1 R_3} \frac{1}{s^2 + \frac{1}{C R_2}s + \frac{1}{\left(C R_3\right)^2}}$$

En este punto podemos notar que se trata de un filtro de segundo orden sin ceros, asi que tiene comportamiento de filtro pasa bajos.
Pasamos la funcion de transferencia a su forma estandard en terminos de $\omega_0$, $Q$ y $K$:

$$T(s) = K \frac{\omega_0^2}{s^2 + \frac{\omega_0}{Q}s + \omega_0^2} $$

Donde:

\begin{align*}
 \omega_0 &= \frac{1}{C R_3}  \\
 Q &= \frac{R_2}{R_3} \\
 K &= - \frac{R_3}{R_1}
\end{align*}

### Verificación

Aprovecho para realizar una verificacion utilizando SymPy modificando el codigo del Trabajo Semanal anterior.

In [1]:
from sympy import solve, symbols, Eq


#Variables conocidas
R1, R2, R3, R4, C, s = symbols("R1 R2 R3 R4 C s")

#Variables desconocidas
H, Vi, Vo, I1, I2, I3, I4, I5, I6, Va, Vb = symbols("H Vi Vo I1 I2 I3 I4 I5 I6 Va Vb")
unknowns = [H, Vi, Vo, I1, I2, I3, I4, I5, I6, Va, Vb]

equations = []

#Ecuaciones de mallas
equations.append(Eq(Vi, I6*R1))
equations.append(Eq(-Va, R2*I1))
equations.append(Eq(-Va, I2*1/(C*s)))
equations.append(Eq(Va, I3*R3))
equations.append(Eq(-Vb, I3*1/(C*s)))
equations.append(Eq(Vb, R4*I4))
equations.append(Eq(-Vo, I4*R4))
equations.append(Eq(Vo, R3*I5))

#Ecuaciones de nodos
equations.append(Eq(I6 + I5, I1 + I2))

#Relacion de H con tensiones
equations.append(Eq(H, Vo/Vi))

#Resolver
sol_H = solve(equations, unknowns, dict=True)[0][H]


sol_H

-R2*R3/(R1*(C**2*R2*R3**2*s**2 + C*R3**2*s + R2))

## 2/3. Elección de componentes

El objetivo propuesto de diseñar el amplificador con parametros $\omega_0 = 1$ y $Q = 3$ sin otra condición nos lleva a inevitablemente elegir un valor para $C$, $R_2$ o $R_3$. Para este diseño propongo $C=1 \mu F$ ya que es un valor comerical muy usado y se pueden adquirir facilmente capacitores no polarizados de este valor. Un valor mas bajo para $C$ puede ser mas conventiente de conseguir, pero nos llevaria a valores de resistencias muy altos y eso podria causar otros problemas en la práctica.

Esto nos lleva a que:


\begin{align*}
 R_3 &= \frac{1}{C \omega_0} = 1M \Omega  \\
 R_2 &= Q R_3 = 3M  \Omega
\end{align*}

Si tambien deseamos cumplir con la condicion de ganancia de continua $\left|T(0)|\right| = 20dB = 10 \times$ debemos evaluar la funcion en $s=0$:

$$|T(0)| = \left|K \frac{\omega_0^2}{0^2 + \frac{\omega_0}{Q} 0^2 + \omega_0^2}\right| = \left|K \omega_0\right| = \left|K\right| = \frac{R_3}{R_1} $$

Lo que buscamos entonces es: $\frac{R_3}{R_1} = 10$. Por lo tanto:

$$R_1 = 100k \Omega$$

Finalmente, el valor de $R_4$ en la teoría no afecta el funcionamiento del circuito, pero decido usar $R_4 = 100k \Omega$ para repetir valores.

En resumen:
\begin{align*}
 R_1 &= R_4 = 100 k \Omega & R_2 &= 3M \Omega & R_3 &= 1M \Omega & C &= 1\mu F
\end{align*}


## 4. Valores de red normalizados

Para normalizar la red  en frecuencia, utilizamos $\Omega_\omega = \omega_0$

Por ende, la variable compleja de frecuencia normalizada resulta ser: $\$ = \frac{s}{\Omega_\omega} = \frac{s}{\omega_0}$

Y para normalizar en impedancia, utilizaremos $\Omega_z = R_3$. Decidí utilizar (de forma heurística) $R_3$ ya que aparece en la expresión de $\omega_0$ y $Q$, entonces es mas probable de que resulte en mejores simplificaciones de otras variables o componentes.

Los valores de los componentes normalizados resultan:

\begin{align*}
 R_1' &= \frac{R_1}{\Omega_z} = -\frac{1}{K}  \\
 R_2' &= \frac{R_2}{\Omega_z} = \frac{Q R_3}{\Omega_z} = Q \\
 R_3' &= \frac{R_3}{\Omega_z} = 1 \\
 R_4' &= \frac{R_4}{\Omega_z}\\
 C'   &= \Omega_\omega \Omega_z C = \frac{\Omega_\omega \Omega_z}{\omega_0 R_3} = 1
\end{align*}

Y la función de transferencia normalizada en frecuencia e impedancia resulta:

$$ T(s) = K \frac{1}{\$^2 + \frac{1}{Q}\$ + 1} $$

Y la representacion gráfica de la red normalizada:

<div>
    <img src="circuito_normalizado.png" width="800"/>
</div>